In [5]:
# 프로젝트 디렉토리에서 실행
# jupyter notebook에서 django shell을 사용할 수 있도록 설정
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'config.settings'
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'

import django
django.setup()

In [6]:
from polls.models import Question, Choice

# 전체조회
- all()

In [4]:
# Model클래스이름.objects.xxx
qs = Question.objects.all() # QuerySet
qs

<QuerySet [<Question: 좋아하는 색깔은 무엇입니까?>, <Question: 가고 싶은 나라를 선택하세요.>]>

In [7]:
# QuerySet.query : sql문 조회
print(qs.query)

SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question"


In [9]:
for q in qs:
    print(q, type(q))

좋아하는 색깔은 무엇입니까? <class 'polls.models.Question'>
가고 싶은 나라를 선택하세요. <class 'polls.models.Question'>


In [10]:
# 특정 조회결과 하나 조회 - indexing
# 범위 조회: slicing
qs[0]

<Question: 좋아하는 색깔은 무엇입니까?>

In [11]:
qs[0:]

[<Question: 좋아하는 색깔은 무엇입니까?>, <Question: 가고 싶은 나라를 선택하세요.>]

In [12]:
qs[-1] # 음수 index는 사용 못함.

AssertionError: Negative indexing is not supported.

In [13]:
qs.first()

<Question: 좋아하는 색깔은 무엇입니까?>

In [14]:
qs.last()

<Question: 가고 싶은 나라를 선택하세요.>

In [15]:
qs = Choice.objects.all()
qs

<QuerySet [<Choice: 검정색>, <Choice: 파란색>, <Choice: 노란색>, <Choice: 미국>, <Choice: 쿠바>]>

In [16]:
qs.order_by('choice_text') # ASC

<QuerySet [<Choice: 검정색>, <Choice: 노란색>, <Choice: 미국>, <Choice: 쿠바>, <Choice: 파란색>]>

In [17]:
qs.order_by('-choice_text') # DESC

<QuerySet [<Choice: 파란색>, <Choice: 쿠바>, <Choice: 미국>, <Choice: 노란색>, <Choice: 검정색>]>

In [18]:
qs = Choice.objects.all().order_by('choice_text')
qs

<QuerySet [<Choice: 검정색>, <Choice: 노란색>, <Choice: 미국>, <Choice: 쿠바>, <Choice: 파란색>]>

In [19]:
print(qs.query)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."vote", "polls_choice"."question_id" FROM "polls_choice" ORDER BY "polls_choice"."choice_text" ASC


In [21]:
c = qs.first()
print(type(c))

<class 'polls.models.Choice'>


In [23]:
# 조회한 Model의 속성을 조회 => 특정 컬럼값 조회 => '.' 표기법
# Choice : id(pk), choice_text, vote, question_id
print(c.id, c.choice_text, c.vote, c.question_id)

1 검정색 2 1


In [24]:
for c in qs:
    print(c.id, c.choice_text, c.vote, c.question_id, sep='\t')

1	검정색	2	1
3	노란색	100	1
4	미국	0	3
5	쿠바	2	3
2	파란색	5	1


# where절
- filter(조회조건)
    - 조회조건을 만족하는 0개 이상의 행을 조회
- exclude(조회조건)
    - filter의 반대. 조회조건을 만족하지 않는 0개 이상의 행을 조회 (not)
- get(조회조건)
    - 조회조건을 만족하는 1개 행을 조회
- filter/exclude : QuerySet으로 반환
- get : Model 객체를 반환

In [31]:
# where
qs = Question.objects.filter(id=3)
# primary key 컬럼 조회: 컬럼명, pk
qs = Question.objects.filter(pk=1)
qs

<QuerySet [<Question: 좋아하는 색깔은 무엇입니까?>]>

In [32]:
print(qs.first().pub_date)

2021-05-25 05:18:45.775911+00:00


In [35]:
quest = Question.objects.get(pk=1) # get(): pk로 조회
print(type(quest))

<class 'polls.models.Question'>


In [36]:
Question.object.get(pk=2)

AttributeError: type object 'Question' has no attribute 'object'

In [37]:
try:
    r = Question.objects.get(pk=1)
    print(r.pub_date)
except:
    print('조회결과가 없습니다.')

2021-05-25 05:18:45.775911+00:00


In [39]:
Question.objects.get(id__gt=1) # id > 1 # 조회결과가 2개 이상일 경우 예외 발생

<Question: 가고 싶은 나라를 선택하세요.>

In [43]:
# 조회조건
# qs = Choice.objects.filter(id__gt=3) # id > 3
qs = Choice.objects.filter(id__gte=3) # id >= 3
for q in qs:
    print(q.id, q.choice_text)

3 노란색
4 미국
5 쿠바


In [54]:
# qs = Question.objects.filter(question_text__startswith='가고') # q_t like '가고%'
# qs = Question.objects.filter(question_text__endswith='하세요.') # q_t like '%하세요.'
qs = Question.objects.filter(question_text__contains='하는') # q_t like '%하는%'
print(qs)
for q in qs:
    print(q.id, q.question_text)

<QuerySet [<Question: 좋아하는 색깔은 무엇입니까?>]>
1 좋아하는 색깔은 무엇입니까?


In [57]:
# qs = Choice.objects.filter(id__in=[2,6,7,20]) # where id in [2,6,7,20]
qs = Choice.objects.filter(id__range=[3,7]) # between 3 and 7
print(qs.query)
for q in qs:
    print(q.id, q.choice_text)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."vote", "polls_choice"."question_id" FROM "polls_choice" WHERE "polls_choice"."id" BETWEEN 3 AND 7
3 노란색
4 미국
5 쿠바


## and조건

In [63]:
qs = Choice.objects.filter(pk=3, choice_text='노란색')
qs.first().id, qs.first().choice_text

(3, '노란색')

In [65]:
qs = Choice.objects.filter(pk__lt=5, choice_text__contains='색')
for q in qs:
    print(q.id, q.choice_text)

1 검정색
2 파란색
3 노란색


In [66]:
qs = Choice.objects.filter(pk__gt=5, choice_text__contains='색')
for q in qs:
    print(q.id, q.choice_text)

In [73]:
# OR조건 Q()
from django.db.models import Q

# qs = Choice.objects.filter(Q(pk__gt=5)|Q(choice_text__contains='색'))
# qs = Choice.objects.filter(Q(pk__gt=5)|~Q(choice_text__contains='색'))
qs = Choice.objects.filter(~Q(pk=5))
for q in qs:
    print(q.id, q.choice_text)

1 검정색
2 파란색
3 노란색
4 미국


## 컬럼선택
select 컬럼, 컬럼
- values(field이름, ...)
    - 개별 조회결과를 Dictionary에 넣어서 반환

In [76]:
qs = Choice.objects.all().values('choice_text', 'vote')
print(qs.query)
for q in qs:
    print(q['choice_text'], q['vote'], q)

SELECT "polls_choice"."choice_text", "polls_choice"."vote" FROM "polls_choice"
검정색 2 {'choice_text': '검정색', 'vote': 2}
파란색 5 {'choice_text': '파란색', 'vote': 5}
노란색 100 {'choice_text': '노란색', 'vote': 100}
미국 0 {'choice_text': '미국', 'vote': 0}
쿠바 2 {'choice_text': '쿠바', 'vote': 2}


# 집계
- 단순 집계
    - aggregate(집계함수(), ...)
- Group by 집계
    - values(그룹으로묶을컬럼).annotate(집계함수(), ...)

In [10]:
# 조회결과 행수 조회
qs = Choice.objects.all()
qs = Choice.objects.filter(id__lt=3)
len(qs), qs.count()

(2, 2)

In [11]:
from django.db.models import Count, Sum, Avg, Min, Max, StdDev, Variance

In [14]:
# dic = Choice.objects.aggregate(Count("id")) # select count('id') from choice
dic = Choice.objects.aggregate(Count('id'), Count('choice_text'), Sum('vote'),
                               Avg('vote'), Min('vote'), StdDev('vote'), Variance('vote'))
print(type(dic))
dic

<class 'dict'>


{'id__count': 10,
 'choice_text__count': 10,
 'vote__sum': 415,
 'vote__avg': 41.5,
 'vote__min': 2,
 'vote__stddev': 41.04692436711915,
 'vote__variance': 1684.85}

In [15]:
dic = Question.objects.aggregate(Min('pub_date'), Max('pub_date'))
dic

{'pub_date__min': datetime.datetime(2021, 5, 25, 5, 18, 45, 775911, tzinfo=<UTC>),
 'pub_date__max': datetime.datetime(2021, 5, 26, 1, 19, 42, 807663, tzinfo=<UTC>)}

In [18]:
print(dic['pub_date__min'])
print(
    dic['pub_date__max'].strftime(
        '%Y년 %m월 %d일'.encode('unicode-escape').decode()
    ).encode().decode('unicode-escape')
)

2021-05-25 05:18:45.775911+00:00
2021년 05월 26일


In [20]:
# where + 집계
dic = Choice.objects.filter(id__gt=5).aggregate(Sum('vote'))
dic

{'vote__sum': 167}

In [21]:
# group by

# select 5
# from 1
# where 2
# group by 3
# having 4
# order by 6

# choice => 질문별 vote의 합계, 평균
dic = Choice.objects.values('question_id').annotate(Sum('vote'), Avg('vote'))
dic

<QuerySet [{'question_id': 1, 'vote__sum': 177, 'vote__avg': 44.25}, {'question_id': 3, 'vote__sum': 123, 'vote__avg': 41.0}, {'question_id': 4, 'vote__sum': 115, 'vote__avg': 38.333333333333336}]>

In [23]:
for d in dic:
    print(f"질문: {d['question_id']}, 투표합계: {d['vote__sum']}, 투표평균: {d['vote__avg']}")

질문: 1, 투표합계: 177, 투표평균: 44.25
질문: 3, 투표합계: 123, 투표평균: 41.0
질문: 4, 투표합계: 115, 투표평균: 38.333333333333336


# 테이블간의 관계

## 자식테이블에서 부모 테이블의 값을 조회

In [26]:
c1 = Choice.objects.get(pk=3)
type(c1), c1  # c1: Choice모델 객체

(polls.models.Choice, <Choice: 노란색>)

In [28]:
# 모델에서 속성 조회 => 각 컬럼 값 조회
c1.id, c1.pk, c1.choice_text, c1.vote

(3, 3, '노란색', 100)

In [31]:
q1 = c1.question # 부모 Model객체
type(q1), q1

(polls.models.Question, <Question: 좋아하는 색깔은 무엇입니까?>)

In [33]:
print(q1.question_text, q1.pub_date)

좋아하는 색깔은 무엇입니까? 2021-05-25 05:18:45.775911+00:00


## 부모테이블에서 자식테이블값을 조회

In [35]:
q = Question.objects.get(pk=3)
type(q), q

(polls.models.Question, <Question: 가고 싶은 나라를 선택하세요.>)

In [36]:
q.id, q.question_text, q.pub_date

(3,
 '가고 싶은 나라를 선택하세요.',
 datetime.datetime(2021, 5, 25, 5, 21, 4, 752190, tzinfo=<UTC>))

In [40]:
c = q.choice_set
type(c)

django.db.models.fields.related_descriptors.create_reverse_many_to_one_manager.<locals>.RelatedManager

In [41]:
c.all()

<QuerySet [<Choice: 미국>, <Choice: 쿠바>, <Choice: 남아공>]>

In [42]:
c.filter(choice_text='한국')

<QuerySet []>

In [43]:
Choice.objects.filter(question_id=3)

<QuerySet [<Choice: 미국>, <Choice: 쿠바>, <Choice: 남아공>]>

# insert/update/delete
- insert/update
    - 모델객체(insert할내용을가진).save()
- delete
    - 모델객체(삭제할pk를가진).delete()

In [45]:
# insert
qs = Question.objects.all()
for q in qs:
    print(q.pk)

1
3
4


In [48]:
from datetime import datetime, timedelta

In [55]:
# pk: id - 자동증가(생략)
new_q = Question()
new_q.id = 2
new_q.question_text = "좋아하는 동물은?"
new_q.pub_date = datetime.now()
new_q.question_text, new_q.pub_date, new_q.pk

('좋아하는 동물은?', datetime.datetime(2021, 5, 26, 11, 24, 2, 2760), 2)

In [56]:
new_q.save()

c:\users\inje.jeong\anaconda3\envs\django\lib\site-packages\django\db\models\fields\__init__.py:1419: RuntimeWarning: DateTimeField Question.pub_date received a naive datetime (2021-05-26 11:24:02.002760) while time zone support is active.
  RuntimeWarning)


In [57]:
qs = Question.objects.all()
for q in qs:
    print(q.pk, q.question_text, q.pub_date)

1 좋아하는 색깔은 무엇입니까? 2021-05-25 05:18:45.775911+00:00
2 좋아하는 동물은? 2021-05-26 02:24:03.438889+00:00
3 가고 싶은 나라를 선택하세요. 2021-05-25 05:21:04.752190+00:00
4 프로젝트는 어떤 주제로 진행했습니까? 2021-05-26 01:19:42.807663+00:00
5 좋아하는 가수는? 2021-05-26 02:22:50.074958+00:00


In [61]:
q = Question.objects.get(pk=1)
q.question_text = "좋아하는 꽃은 무엇입니까?"
q.pub_date = datetime.now() - timedelta(days=100)
print(q.id, q.question_text, q.pub_date)
q.save()

1 좋아하는 꽃은 무엇입니까? 2021-02-15 11:30:03.527117


c:\users\inje.jeong\anaconda3\envs\django\lib\site-packages\django\db\models\fields\__init__.py:1419: RuntimeWarning: DateTimeField Question.pub_date received a naive datetime (2021-02-15 11:30:03.527117) while time zone support is active.
  RuntimeWarning)


In [62]:
qs = Question.objects.all()
for q in qs:
    print(q.pk, q.question_text, q.pub_date)

1 좋아하는 꽃은 무엇입니까? 2021-02-15 02:30:03.527117+00:00
2 좋아하는 동물은? 2021-05-26 02:24:03.438889+00:00
3 가고 싶은 나라를 선택하세요. 2021-05-25 05:21:04.752190+00:00
4 프로젝트는 어떤 주제로 진행했습니까? 2021-05-26 01:19:42.807663+00:00
5 좋아하는 가수는? 2021-05-26 02:22:50.074958+00:00


In [63]:
# 삭제
d_q = Question()
d_q.id = 1
d_q.id, d_q.question_text, d_q.pub_date

(1, '', None)

In [64]:
d_q.delete()

(5, {'polls.Choice': 4, 'polls.Question': 1})

In [65]:
qs = Question.objects.all()
for q in qs:
    print(q.pk, q.question_text, q.pub_date)

2 좋아하는 동물은? 2021-05-26 02:24:03.438889+00:00
3 가고 싶은 나라를 선택하세요. 2021-05-25 05:21:04.752190+00:00
4 프로젝트는 어떤 주제로 진행했습니까? 2021-05-26 01:19:42.807663+00:00
5 좋아하는 가수는? 2021-05-26 02:22:50.074958+00:00


In [67]:
Choice.objects.all()

<QuerySet [<Choice: 미국>, <Choice: 쿠바>, <Choice: 프론트엔드>, <Choice: 백엔드>, <Choice: AI>, <Choice: 남아공>]>

In [70]:
# 특정 조건의 데이터들을 삭제 => 조회
# choice_text에 '색'이 들어간 데이터 삭제
qs = Choice.objects.filter(choice_text__endswith='색')
for q in qs:
    q.delete()

In [71]:
Choice.objects.all()

<QuerySet [<Choice: 미국>, <Choice: 쿠바>, <Choice: 프론트엔드>, <Choice: 백엔드>, <Choice: AI>, <Choice: 남아공>]>

# 직접 SQL문 실행
- 모델.objects.raw('select문')
    - RawQuerySet

In [75]:
rq = Choice.objects.raw('select * from polls_choice')
print(rq.query)

select * from polls_choice


In [76]:
for q in rq:
    print(q, type(q))

미국
쿠바
프론트엔드
백엔드
AI
남아공
